In [8]:
import grpc
import sys
from concurrent import futures
import inference_pb2_grpc 
import onnx
import onnxruntime
import numpy as np
import inference_pb2 
import ezkl
import ast
import config
import ecdsa
from ecdsa.keys import SigningKey, VerifyingKey
import hashlib

In [2]:
class InferenceServer(inference_pb2_grpc.InferenceServicer):
    def RunInference(self, inferenceParams, context):
        results = self.Infer(inferenceParams.modelHash, inferenceParams.modelInput)
        return inference_pb2.InferenceResult(tx=inferenceParams.tx, node=inferenceParams.modelHash, value=str(results))
    
    def Infer(self, modelHash, modelInput):
        session = onnxruntime.InferenceSession(modelHash)
        results = session.run(curateOutputs(session), curateInputs(session, modelInput))[-1]
        return results[0][0]

In [9]:
def serve(port, maxWorkers):
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=maxWorkers))
    inference_pb2_grpc.add_InferenceServicer_to_server(InferenceServer(), server)
    server.add_insecure_port("[::]:" + str(port))
    server.start()
    server.wait_for_termination()
    
def parseInput(modelInput, typeString):
    if "tensor" in typeString:
        if "float" in typeString:
            return onnxruntime.OrtValue.ortvalue_from_numpy(np.array([modelInput]).astype("float32"))
        if "string" in typeString:
            return onnxruntime.OrtValue.ortvalue_from_numpy(np.array([modelInput]).astype("string"))

def curateInputs(session, modelInput):
    inputs = {}
    sessionInputs = session.get_inputs()
    for i in range(0, len(sessionInputs)):
        param = ast.literal_eval(modelInput)[i]
        inputs[sessionInputs[i].name] = parseInput(param, sessionInputs[i].type)
    return inputs

def curateOutputs(session):
    outputs = []
    for o in session.get_outputs():
        outputs.append(o.name)
    return outputs

def sign(private_key):
    signing_key = ecdsa.SigningKey.from_string(private_key, curve=ecdsa.SECP256k1)

In [5]:
serve(config.port=5125, config.maxWorkers=100)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-9626c6f82927>", line 1, in <module>
    serve(port=5125, maxWorkers=100)
  File "<ipython-input-3-dea5067263b6>", line 6, in serve
    server.wait_for_termination()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/grpc/_server.py", line 1118, in wait_for_termination
    return _common.wait(self._state.termination_event.wait,
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/grpc/_common.py", line 150, in wait
    _wait_once(wait_fn, MAXIMUM_WAIT_TIMEOUT, spin_cb)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/grpc/_common.py", line 112, in _wait_once
    wait_fn(timeout=timeout)
  File "/Library/Frameworks/Python.framework

KeyboardInterrupt: 